In [1]:
import numpy as np 
import pandas as pd 
import plotly.express as px
import os

In [2]:
print(f"cwd : {os.getcwd()}")

cwd : /Users/imantha/workspace/cryo-polygen/cryo-polygen/energy-equipment/experiments/experiment_2


## Data
- data1_280722
- explanatory variables : "xk","xpl","xmw","xTe"
- response variables : "yts", "ytt"

In [3]:
data_path = os.path.join("..","..","data","data1_280722.csv")
df = pd.read_csv(data_path)
# Remove NaN columns
df.drop(["Unnamed: 4", "Unnamed: 5"], axis = 1, inplace = True)
# Remove any missing values
df.dropna(inplace=True)

df.head()

print(f"df size : {df.shape}")

df size : (4563, 6)


In [4]:
p = px.scatter_matrix(df, color = "ytt", color_continuous_scale="Sunsetdark")
p.update_layout(template="plotly_white")

## Base Model

In [5]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, KFold, cross_validate, RandomizedSearchCV, GridSearchCV

from sklearn.svm import SVR
from sklearn.multioutput import MultiOutputRegressor

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

from scipy import stats

In [6]:
X = df.drop(["yts","ytt"],axis = 1).values
y = df[["yts","ytt"]].values

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.2)
print(f"X_train : {X_train.shape}, y_train : {y_train.shape}")
print(f"X_test : {X_test.shape}, y_test : {y_test.shape}")

X_train : (3650, 4), y_train : (3650, 2)
X_test : (913, 4), y_test : (913, 2)


## Grid / Randomised Search

In [14]:
def train_model(param_grid, estimator,estimator_name,train_set):
    # MultiOutputRegressor(estimator = SVR())
    # Get X_train, y_train
    X_train, y_train = train_set

    # Construct pipeline
    pipe = Pipeline([
        ("scaler", MinMaxScaler()),
        (estimator_name, estimator)
    ])

    # Randomized Search
    search = RandomizedSearchCV(
        pipe,
        param_grid,
        scoring = ("neg_mean_squared_error"),
        cv = KFold(n_splits = 5, shuffle = True),
        n_jobs = -1,
        refit = True,
        n_iter = 60,
        verbose = 0,
        return_train_score = True
    )
    
    # Fit model
    search.fit(X_train, y_train)

    return search

def get_cv_result(model):
    results = pd.DataFrame(model.cv_results_)
    results.sort_values(by = "mean_test_score", ascending=False, inplace=True)
    
    p = px.scatter(
        x = np.arange(0,results.shape[0]), 
        y = results["mean_test_score"],
        error_y=results["std_test_score"]
    )
    p.update_layout(
        template = "plotly_white", 
        xaxis_title = "hyperparam_idx", 
        yaxis_title = "MSE"
    )

    return results, p

def get_predictions_and_scores(model,model_type,predictions, metrics):
    yhat = model.predict(X)
    predictions[[f"yts_hat_{model_type}",f"ytt_hat_{model_type}"]] = yhat

    rmse = np.sqrt(mean_squared_error(y, yhat))
    mae = mean_absolute_error(y, yhat)
    r2 = r2_score(y, yhat)
    var = np.var(y - yhat)

    model_metrics = pd.DataFrame({
        "model" : [model_type],
        "rmse" : [rmse],
        "mae" : [mae],
        "r2" : [r2],
        "var" : [np.var(y - yhat)]
    })
    metrics = pd.concat([metrics, model_metrics],axis = 0)

    return predictions, metrics

# Empty DataFrame to store predictions and metrics

In [32]:
# Store Predictions
predictions = pd.DataFrame(columns = ["yts", "ytt"])
predictions[["yts","ytt"]] = y

# Store Metrics
metrics = pd.DataFrame(columns = ["model","rmse", "mae", "r2", "var"])

# SVM

In [33]:
# Define parameters space
param_grid = [
    {
        "svm__estimator__C" : stats.uniform(0.1,100), 
        "svm__estimator__kernel" : ["linear"]
    },
    {
        "svm__estimator__C" : stats.loguniform(0.1, 100), 
        "svm__estimator__gamma" : stats.loguniform(0.0001, 10), 
        "svm__estimator__kernel" : ["poly","rbf"]
    }
]

search = train_model(
    param_grid=param_grid, 
    estimator = MultiOutputRegressor(estimator = SVR()),
    estimator_name = "svm",
    train_set=(X_train, y_train)
)

print(f"Best Params : {search.best_params_}")
print(f"Best train score : {search.best_score_}")

cv_results, p = get_cv_result(search)
#print(results.head())
p.show()

Best Params : {'svm__estimator__C': 4.4022721550394035, 'svm__estimator__gamma': 3.5050208354562353, 'svm__estimator__kernel': 'rbf'}
Best train score : -1.6727467176094997


In [34]:
predictions, metrics = get_predictions_and_scores(search.best_estimator_, "svm", predictions, metrics)
print(predictions.head())
print(metrics.head())

   yts  ytt  yts_hat_svm  ytt_hat_svm
0  0.0  0.0    -0.087126     0.286691
1  0.0  0.5     0.122054     0.399489
2  0.0  1.0     0.516024     0.550245
3  0.0  1.5     1.044092     0.690250
4  0.0  2.0     1.665306     0.824535
  model      rmse       mae        r2       var
0   svm  1.282204  0.993668  0.530272  1.643202


In [35]:
predictions.to_csv("results/prediction.csv")
metrics.to_csv("results/metrics.csv")

# Random Forest

In [36]:
from sklearn.ensemble import RandomForestRegressor

param_grid = {
"rf__max_depth" : np.arange(10,120,10),
"rf__n_estimators" : np.arange(100,1000, 100),
"rf__min_samples_split" : stats.randint(1,10),
"rf__min_samples_leaf" : stats.randint(1,5)
}

search = train_model(
    param_grid=param_grid, 
    estimator = RandomForestRegressor(),
    estimator_name = "rf",
    train_set=(X_train, y_train)
)

print(f"Best Params : {search.best_params_}")
print(f"Best train score : {search.best_score_}")

# Cv results
results, p = get_cv_result(search)
results = results[["param_rf__max_depth","param_rf__n_estimators","param_rf__min_samples_split","param_rf__min_samples_leaf","mean_train_score","std_train_score"]]
#print(results.head())
p.show()

/Users/imantha/Software/miniforge3/envs/ml3.10/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning:


25 fits failed out of a total of 300.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
25 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/imantha/Software/miniforge3/envs/ml3.10/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/imantha/Software/miniforge3/envs/ml3.10/lib/python3.10/site-packages/sklearn/pipeline.py", line 394, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/Users/imantha/Software/miniforge3/envs/ml3.10/lib

Best Params : {'rf__max_depth': 10, 'rf__min_samples_leaf': 1, 'rf__min_samples_split': 9, 'rf__n_estimators': 200}
Best train score : -2.031651252455668


In [37]:
predictions, metrics = get_predictions_and_scores(search.best_estimator_, "rf", predictions, metrics)
print(predictions.head())
print(metrics.head())

   yts  ytt  yts_hat_svm  ytt_hat_svm  yts_hat_rf  ytt_hat_rf
0  0.0  0.0    -0.087126     0.286691    0.000000    0.000000
1  0.0  0.5     0.122054     0.399489    0.078356    0.432029
2  0.0  1.0     0.516024     0.550245    0.594105    0.594534
3  0.0  1.5     1.044092     0.690250    0.952955    0.898765
4  0.0  2.0     1.665306     0.824535    1.806837    0.835528
  model      rmse       mae        r2       var
0   svm  1.282204  0.993668  0.530272  1.643202
0    rf  1.189312  0.914096  0.595867  1.414463


# KNN

In [38]:
from sklearn.neighbors import KNeighborsRegressor

param_grid = {
    "knn__n_neighbors" : [3,5,7,9,11,13,15],
    "knn__weights" : ["uniform", "distance"],
    "knn__metric" : ["minkowski","euclidean","manhattan"]
}

search = train_model(
    param_grid = param_grid, 
    estimator = KNeighborsRegressor(),
    estimator_name = "knn",
    train_set=(X_train, y_train)
)

print(f"Best Params : {search.best_params_}")
print(f"Best train score : {search.best_score_}")

# Cv results
results, p = get_cv_result(search)
results = results[["param_knn__n_neighbors","param_knn__weights","param_knn__metric","mean_train_score","std_train_score"]]
#print(results.head())
p.show()

/Users/imantha/Software/miniforge3/envs/ml3.10/lib/python3.10/site-packages/sklearn/model_selection/_search.py:292: UserWarning:

The total space of parameters 42 is smaller than n_iter=60. Running 42 iterations. For exhaustive searches, use GridSearchCV.



Best Params : {'knn__weights': 'uniform', 'knn__n_neighbors': 15, 'knn__metric': 'manhattan'}
Best train score : -1.8260555555555555


In [39]:
predictions, metrics = get_predictions_and_scores(search.best_estimator_, "knn", predictions, metrics)
print(predictions.head())
print(metrics.head())

   yts  ytt  yts_hat_svm  ytt_hat_svm  yts_hat_rf  ytt_hat_rf  yts_hat_knn  \
0  0.0  0.0    -0.087126     0.286691    0.000000    0.000000     0.466667   
1  0.0  0.5     0.122054     0.399489    0.078356    0.432029     0.533333   
2  0.0  1.0     0.516024     0.550245    0.594105    0.594534     0.933333   
3  0.0  1.5     1.044092     0.690250    0.952955    0.898765     1.200000   
4  0.0  2.0     1.665306     0.824535    1.806837    0.835528     1.666667   

   ytt_hat_knn  
0     0.400000  
1     0.433333  
2     0.533333  
3     0.700000  
4     0.966667  
  model      rmse       mae        r2       var
0   svm  1.282204  0.993668  0.530272  1.643202
0    rf  1.189312  0.914096  0.595867  1.414463
0   knn  1.289251  1.003664  0.525095  1.662167


# Elastic Net

In [42]:
from sklearn.linear_model import ElasticNet

param_grid = {
    "glm__alpha" : stats.uniform(1e-5,100),
    "glm__l1_ratio" : np.arange(0,1,0.01),
}

search = train_model(
    param_grid = param_grid, 
    estimator = ElasticNet(),
    estimator_name = "glm",
    train_set=(X_train, y_train)
)

print(f"Best Params : {search.best_params_}")
print(f"Best train score : {search.best_score_}")

# Cv results
results, p = get_cv_result(search)
#results = results[["param_knn__n_neighbors","param_knn__weights","param_knn__metric","mean_train_score","std_train_score"]]
#print(results.head())
p.show()

Best Params : {'glm__alpha': 5.8494253887981955, 'glm__l1_ratio': 0.0}
Best train score : -3.4440650356074003


/Users/imantha/Software/miniforge3/envs/ml3.10/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.072e+03, tolerance: 1.016e+00 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(
/Users/imantha/Software/miniforge3/envs/ml3.10/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.042e+03, tolerance: 1.011e+00 Linear regression models with null weight for the l1 regularization term are more effici

In [43]:
predictions, metrics = get_predictions_and_scores(search.best_estimator_, "glm", predictions, metrics)
print(predictions.head())
print(metrics.head())

   yts  ytt  yts_hat_svm  ytt_hat_svm  yts_hat_rf  ytt_hat_rf  yts_hat_knn  \
0  0.0  0.0    -0.087126     0.286691    0.000000    0.000000     0.466667   
1  0.0  0.5     0.122054     0.399489    0.078356    0.432029     0.533333   
2  0.0  1.0     0.516024     0.550245    0.594105    0.594534     0.933333   
3  0.0  1.5     1.044092     0.690250    0.952955    0.898765     1.200000   
4  0.0  2.0     1.665306     0.824535    1.806837    0.835528     1.666667   

   ytt_hat_knn  yts_hat_glm  ytt_hat_glm  
0     0.400000     2.974147     2.931261  
1     0.433333     2.977766     2.939093  
2     0.533333     2.981354     2.946854  
3     0.700000     2.984845     2.954408  
4     0.966667     2.988304     2.961892  
  model      rmse       mae        r2       var
0   svm  1.282204  0.993668  0.530272  1.643202
0    rf  1.189312  0.914096  0.595867  1.414463
0   knn  1.289251  1.003664  0.525095  1.662167
0   glm   1.85968  1.606516  0.011883  3.458395


## ANN

In [44]:
import os 
import sys
import torch 
import torch.nn as nn 
import torch.optim as optim 
import torch.nn.functional as F

from copy import deepcopy

In [45]:
X = df.drop(["yts","ytt"],axis = 1).values
y = df[["yts","ytt"]].values
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.2)
X_train, X_val,y_train, y_val = train_test_split(X_train, y_train, test_size=0.2)

print(f"X_train : {X_train.shape}, y_train : {y_train.shape}")
print(f"X_val : {X_val.shape}, y_train : {y_val.shape}")
print(f"X_test : {X_test.shape}, y_test : {y_test.shape}")


X_train : (2920, 4), y_train : (2920, 2)
X_val : (730, 4), y_train : (730, 2)
X_test : (913, 4), y_test : (913, 2)


In [46]:
class Simple_MLP(nn.Module):
    def  __init__(self, l1, l2):
        super(Simple_MLP, self).__init__()
        self.fc1 = nn.Linear(in_features= 4, out_features=l1)
        self.fc2 = nn.Linear(in_features = l1, out_features = l2)
        self.fc3 = nn.Linear(in_features=l2, out_features = 2) 

    def forward(self, X):
        out = F.tanh(self.fc1(X))
        out = F.tanh(self.fc2(out))
        out = self.fc3(out)
        return out


In [47]:
def train_MLP(model,criterion, optimizer,data,epochs=100):
    training_loss = []
    validation_loss = []

    X_train, X_val, y_train, y_val = data

    min_val_loss = sys.maxsize

    for e in range(epochs):
        # forward pass
        model.train()
        yhat_train = model.forward(X_train)
        loss = criterion(yhat_train, y_train)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        with torch.no_grad():
            model.eval()
            yhat_val = model.forward(X_val)   
            val_loss = criterion(yhat_val, y_val)
            training_loss.append(loss.item())
            validation_loss.append(val_loss.item())

        if val_loss.item() < min_val_loss:
            min_val_loss = val_loss.item()
            best_model = deepcopy(model)

        if e < 10 or (e > 10 and e%100 == 0):
            print(f"epochs : {e}, train_loss : {loss.item()}, val_loss : {val_loss.item()}")

    return best_model


In [48]:
X_train_t = torch.Tensor(X_train)
X_val_t = torch.Tensor(X_val)
X_test_t = torch.Tensor(X_test)
y_train_t = torch.Tensor(y_train)
y_val_t = torch.Tensor(y_val)
y_test_t = torch.tensor(y_test)

print(f"X_train : {X_train_t.shape}, y_train : {y_train_t.shape}")
print(f"X_val : {X_val_t.shape}, y_train : {y_val_t.shape}")
print(f"X_test : {X_test_t.shape}, y_test : {y_test_t.shape}")

X_train : torch.Size([2920, 4]), y_train : torch.Size([2920, 2])
X_val : torch.Size([730, 4]), y_train : torch.Size([730, 2])
X_test : torch.Size([913, 4]), y_test : torch.Size([913, 2])


In [49]:
model = Simple_MLP(l1 = 256, l2 = 128)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters())
data = (X_train_t, X_val_t, y_train_t, y_val_t)

best_model = train_MLP(model, criterion, optimizer, data, 10000)

/Users/imantha/Software/miniforge3/envs/ml3.10/lib/python3.10/site-packages/torch/nn/functional.py:1795: UserWarning:

nn.functional.tanh is deprecated. Use torch.tanh instead.



epochs : 0, train_loss : 12.22513198852539, val_loss : 8.722476959228516
epochs : 1, train_loss : 7.987221717834473, val_loss : 5.780328273773193
epochs : 2, train_loss : 5.344764232635498, val_loss : 4.211394309997559
epochs : 3, train_loss : 4.020527362823486, val_loss : 3.5644333362579346
epochs : 4, train_loss : 3.5599913597106934, val_loss : 3.4145798683166504
epochs : 5, train_loss : 3.5438458919525146, val_loss : 3.471426248550415
epochs : 6, train_loss : 3.691584825515747, val_loss : 3.576842784881592
epochs : 7, train_loss : 3.8551723957061768, val_loss : 3.662606716156006
epochs : 8, train_loss : 3.974997043609619, val_loss : 3.709141254425049
epochs : 9, train_loss : 4.038032531738281, val_loss : 3.7177810668945312
epochs : 100, train_loss : 3.430795192718506, val_loss : 3.3611128330230713
epochs : 200, train_loss : 2.4332075119018555, val_loss : 2.3196890354156494
epochs : 300, train_loss : 2.0114951133728027, val_loss : 1.9164535999298096
epochs : 400, train_loss : 1.96055

In [51]:
best_model.forward(torch.Tensor(X))

/Users/imantha/Software/miniforge3/envs/ml3.10/lib/python3.10/site-packages/torch/nn/functional.py:1795: UserWarning:

nn.functional.tanh is deprecated. Use torch.tanh instead.



tensor([[0.0233, 0.0358],
        [0.3380, 0.2716],
        [0.7066, 0.5387],
        ...,
        [3.8375, 5.5564],
        [4.5527, 5.7813],
        [5.3837, 5.9800]], grad_fn=<AddmmBackward0>)

In [56]:
def get_ann_predictions_and_scores(model,model_type,predictions, metrics):
    yhat = model.forward(torch.Tensor(X)).detach().numpy()
    predictions[[f"yts_hat_{model_type}",f"ytt_hat_{model_type}"]] = yhat

    rmse = np.sqrt(mean_squared_error(y, yhat))
    mae = mean_absolute_error(y, yhat)
    r2 = r2_score(y, yhat)
    var = np.var(y - yhat)

    model_metrics = pd.DataFrame({
        "model" : [model_type],
        "rmse" : [rmse],
        "mae" : [mae],
        "r2" : [r2],
        "var" : [np.var(y - yhat)]
    })
    metrics = pd.concat([metrics, model_metrics],axis = 0)

    return predictions, metrics

In [57]:
predictions, metrics = get_ann_predictions_and_scores(best_model, "mlp", predictions, metrics)

/Users/imantha/Software/miniforge3/envs/ml3.10/lib/python3.10/site-packages/torch/nn/functional.py:1795: UserWarning:

nn.functional.tanh is deprecated. Use torch.tanh instead.



In [58]:
predictions

,yts,ytt,yts_hat_svm,ytt_hat_svm,yts_hat_rf,ytt_hat_rf,yts_hat_knn,ytt_hat_knn,yts_hat_glm,ytt_hat_glm,yts_hat_mlp,ytt_hat_mlp
0,0.0,0.0,-0.087126,0.286691,0.000000,0.000000,0.466667,0.400000,2.974147,2.931261,0.023348,0.035791
1,0.0,0.5,0.122054,0.399489,0.078356,0.432029,0.533333,0.433333,2.977766,2.939093,0.337959,0.271633
2,0.0,1.0,0.516024,0.550245,0.594105,0.594534,0.933333,0.533333,2.981354,2.946854,0.706614,0.538725
3,0.0,1.5,1.044092,0.690250,0.952955,0.898765,1.200000,0.700000,2.984845,2.954408,1.127121,0.789690
4,0.0,2.0,1.665306,0.824535,1.806837,0.835528,1.666667,0.966667,2.988304,2.961892,1.626213,1.021850
...,...,...,...,...,...,...,...,...,...,...,...,...
4558,6.0,4.0,3.802933,4.717849,4.109609,4.718179,3.633333,4.766667,3.010238,3.003729,3.209675,4.907147
4559,6.0,4.5,4.007503,5.107316,4.189831,5.118662,3.800000,5.033333,3.011948,3.007427,3.387018,5.266354
4560,6.0,5.0,4.303205,5.454994,3.439268,5.714574,3.966667,5.533333,3.013708,3.011235,3.837537,5.556438
4561,6.0,5.5,4.691226,5.751116,5.070110,5.758485,4.266667,5.766667,3.015538,3.015196,4.552730,5.781271


In [59]:
metrics

,model,rmse,mae,r2,var
0,svm,1.282204,0.993668,0.530272,1.643202
0,rf,1.189312,0.914096,0.595867,1.414463
0,knn,1.289251,1.003664,0.525095,1.662167
0,glm,1.85968,1.606516,0.011883,3.458395
0,mlp,1.275147,0.99475,0.535429,1.625739


# Save DataFrames

In [60]:
predictions.to_csv("results/prediction.csv")
metrics.to_csv("results/metrics.csv")